In [7]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Direct path to project root
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, PROJECT_ROOT)

# Import paths
from src.paths import PROCESSED_AUDIO_DIR, PLOTS_DIR, MFCC_PLOTS_DIR, SPECTRAL_PLOTS_DIR


In [8]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Ensure project root is in path
PROJECT_ROOT = r"c:\Users\iamsh\OneDrive\Desktop\CHI 2026\DRI_SWAASA-main"
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.paths import PROCESSED_AUDIO_DIR, PLOTS_DIR, MFCC_PLOTS_DIR, SPECTRAL_PLOTS_DIR

# Load CSV
csv_path = os.path.join(PROCESSED_AUDIO_DIR, "extracted_features.csv")
df = pd.read_csv(csv_path)

# Quick check
print("Number of audio files in CSV:", len(df))
df.head()

Number of audio files in CSV: 10


,filename,recording_length,rms_power,zero_crossing_rate,crest_factor,mfcc_1_mean,mfcc_1_std,mfcc_2_mean,mfcc_2_std,mfcc_3_mean,...,mfcc_13_std,spectral_centroid,spectral_rolloff,spectral_spread,spectral_flatness,spectral_skewness,spectral_kurtosis,spectral_std,spectral_slope,spectral_decrease
0,C01.mp3,10.344490,0.038089,0.135228,14.321193,-241.30396,91.666275,84.142680,21.002813,-4.462087,...,6.532364,2704.818597,6107.294385,2786.489230,0.051419,16.852112,348.250119,21.021013,-0.001677,0.006595
1,C02.mp3,8.646531,0.025775,0.125209,16.920517,-327.70230,119.825050,91.567764,26.206709,-8.425058,...,6.866963,2444.247799,5257.334429,2619.219154,0.034675,8.877528,101.385738,6.324075,-0.000770,0.004322
2,C03.mp3,13.818776,0.063215,0.094785,9.508083,-213.03752,89.191410,116.747574,28.595161,-19.327377,...,7.169479,1897.756926,3703.765132,2121.751823,0.021194,5.990745,40.733205,31.311410,-0.003855,0.017557
3,C04.mp3,10.475102,0.093060,0.081651,7.570521,-213.18924,118.971700,124.040565,31.689281,-15.035368,...,6.932873,1749.728188,3425.756350,2105.044723,0.011045,5.914286,38.410436,64.781080,-0.007476,0.020954
4,C05.mp3,10.135510,0.089877,0.078488,8.532025,-199.17240,93.771120,126.659700,25.368902,-24.815289,...,8.760152,1683.891182,3190.338679,2068.883210,0.006417,5.443716,31.609686,38.163662,-0.004678,0.021986


In [9]:
# Paths already imported in the first cell

In [10]:
spectral_cols = [
    "spectral_centroid",
    "spectral_rolloff",
    "spectral_spread",
    "spectral_flatness",
    "spectral_skewness",
    "spectral_kurtosis",
    "spectral_std",
    "spectral_slope",
    "spectral_decrease"
]

for filename in df['filename']:
    row = df[df["filename"] == filename]

    spectral_vals = row[spectral_cols].values.flatten()

    plt.figure(figsize=(10,5))
    plt.plot(spectral_cols, spectral_vals, marker='o', linestyle='-', color='lightgreen')
    plt.xticks(rotation=90)
    desc = ("This plot summarizes core spectral descriptors for the selected audio file.\n"
            "Peaks indicate features with stronger frequency-domain prominence.\n"
            "The profile helps compare cough energy distribution across recordings.")
    plt.figtext(0.5, 0.01, desc, ha='center', fontsize=9, wrap=True)
    plt.tight_layout(rect=[0, 0.13, 1, 1])
    plt.title(f"Spectral Features – {filename}")
    plt.xlabel("Feature")
    plt.ylabel("Value")

    # Save plot
    out_path = os.path.join(SPECTRAL_PLOTS_DIR, f"{filename}_spectral_features.png")
    plt.savefig(out_path)
    plt.close()


In [11]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import skew, kurtosis
from src.paths import RAW_AUDIO_DIR


for filename in df['filename']:

    audio_path = os.path.join(RAW_AUDIO_DIR, filename)
    y, sr = librosa.load(audio_path, sr=None)

    # STFT magnitude
    S = np.abs(librosa.stft(y))
    freqs = librosa.fft_frequencies(sr=sr)

    # ---- Core spectral features ----
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
    flatness = librosa.feature.spectral_flatness(y=y)[0]

    # ---- Derived spectral features ----
    spread = np.sqrt(
        np.sum(((freqs[:, None] - centroid)**2) * S, axis=0) / np.sum(S, axis=0)
    )

    spectral_std = np.std(S, axis=0)

    spectral_skewness = skew(S, axis=0)
    spectral_kurtosis = kurtosis(S, axis=0)

    # Spectral slope
    slope = []
    for i in range(S.shape[1]):
        slope.append(np.polyfit(freqs, S[:, i], 1)[0])
    slope = np.array(slope)

    # Spectral decrease
    decrease = []
    for i in range(S.shape[1]):
        numerator = np.sum((S[1:, i] - S[0, i]) / np.arange(1, len(S[:, i])))
        denominator = np.sum(S[1:, i])
        decrease.append(numerator / denominator if denominator != 0 else 0)
    decrease = np.array(decrease)

    # Store features
    feature_dict = {
        "spectral_centroid": centroid,
        "spectral_rolloff": rolloff,
        "spectral_spread": spread,
        "spectral_flatness": flatness,
        "spectral_skewness": spectral_skewness,
        "spectral_kurtosis": spectral_kurtosis,
        "spectral_std": spectral_std,
        "spectral_slope": slope,
        "spectral_decrease": decrease
    }

    # ---- Plot each feature ----
    for feature_name, values in feature_dict.items():

        frames = np.arange(len(values))

        plt.figure(figsize=(10,5))
        plt.plot(frames, values)
        plt.title(f"{filename} - {feature_name}")
        plt.xlabel("Frames")
        plt.ylabel("Value")
        desc = (f"This frame-wise plot shows temporal variation of {feature_name.replace('_', ' ')}.\n"
                "Sharp rises or drops indicate abrupt acoustic changes in the signal.\n"
                "It helps inspect how the spectral characteristic evolves during the cough.")
        plt.figtext(0.5, 0.01, desc, ha='center', fontsize=9, wrap=True)
        plt.tight_layout(rect=[0, 0.13, 1, 1])

        out_path = os.path.join(SPECTRAL_PLOTS_DIR, f"{filename}_{feature_name}.png")
        plt.savefig(out_path)
        plt.close()